In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import os
from utils import utils
import re

os.environ["CUDA_VISIBLE_DEVICES"] = "8"

INFO 05-05 22:13:24 __init__.py:190] Automatically detected platform cuda.


In [2]:
model_name = "./data/models/Qwen3-8B"
model = LLM(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

INFO 05-05 22:13:38 config.py:542] This model supports multiple tasks: {'reward', 'generate', 'score', 'embed', 'classify'}. Defaulting to 'generate'.
WARNING 05-05 22:13:38 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 05-05 22:13:38 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 05-05 22:13:38 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='./data/models/Qwen3-8B', speculative_config=None, tokenizer='./data/models/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 05-05 22:13:43 model_runner.py:1115] Loading model weights took 15.2585 GB
INFO 05-05 22:13:44 worker.py:267] Memory profiling takes 0.77 seconds
INFO 05-05 22:13:44 worker.py:267] the current vLLM instance can use total_gpu_memory (47.41GiB) x gpu_memory_utilization (0.90) = 42.67GiB
INFO 05-05 22:13:44 worker.py:267] model weights take 15.26GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 25.95GiB.
INFO 05-05 22:13:44 executor_base.py:110] # CUDA blocks: 11811, # CPU blocks: 1820
INFO 05-05 22:13:44 executor_base.py:115] Maximum concurrency for 40960 tokens per request: 4.61x
INFO 05-05 22:13:46 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:16<00:00,  2.14it/s]

INFO 05-05 22:14:02 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.71 GiB
INFO 05-05 22:14:02 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 18.89 seconds


In [4]:
math_500 = utils.read_from_json("./data/proved_Math-500.json")

In [6]:
prompt = """Solve the following question Your answer should be in \\boxed\{\} format.

Here is the question:
""" + math_500[0]["problem"] 
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True, 
    enable_thinking=True
)

print(text)

<|im_start|>user
Solve the following question Your answer should be in \boxed\{\} format.

Here is the question:
Compute $58_9 - 18_9.$ Express your answer in base $9.$<|im_end|>
<|im_start|>assistant



In [7]:
sampling_params = SamplingParams(temperature=0.6, top_p=0.95, max_tokens=8096)
output = model.generate(text, sampling_params=sampling_params)
output_text = output[0].outputs[0].text
print(output_text)

Processed prompts: 100%|██████████| 1/1 [00:55<00:00, 55.07s/it, est. speed input: 0.93 toks/s, output: 36.05 toks/s]

<think>
Okay, so I need to subtract two numbers in base 9: 58 base 9 minus 18 base 9. Hmm, let me think. First, I remember that when subtracting numbers in different bases, it's often easier to convert them to base 10, do the subtraction there, and then convert back to the original base. But maybe I can do it directly in base 9? Let me try both methods and see which is easier.

Starting with converting to base 10. Let me recall how base conversion works. For base 9, each digit represents a power of 9. So, for the number 58_9, the rightmost digit is 8 * 9^0, and the next one is 5 * 9^1. Let me calculate that.

First, 58_9:
5 * 9^1 + 8 * 9^0 = 5*9 + 8*1 = 45 + 8 = 53 in base 10.

Then, 18_9:
1 * 9^1 + 8 * 9^0 = 1*9 + 8*1 = 9 + 8 = 17 in base 10.

So subtracting them in base 10: 53 - 17 = 36. Now, I need to convert 36 back to base 9.

To convert 36 to base 9, I need to find how many times 9^1 (which is 9) goes into 36, and then the remainder.

Divide 36 by 9: 36 / 9 = 4 with a remainder o

In [8]:
matches = re.findall(r'\\boxed\{(.*?)\}', output_text)

if matches:
    last_boxed = matches[-1]
    print(last_boxed)

40_9
